In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#from sklearn.multioutput import MultiOutputRegressor
from src.algorithms.multioutput import MultiOutputRegressor, RegressorChain

from sklearn.utils import shuffle
#from catboost import CatBoostRegressor
from src.algorithms.catboost_model import CatBoost

from sklearn.metrics import mean_absolute_error

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

import src.data as data
import src.utility as utility

In [3]:
speeds = data.speeds_original()
events = data.events()
sensors = data.sensors()
merged = utility.merge_speed_events(speeds, events)
merged = pd.merge(merged, sensors, on=['KEY', 'KM'])

caching resources/dataset/originals/speeds_train.csv.gz
caching resources/dataset/preprocessed/events_train.csv.gz


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3209: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



caching sensors

sensors loaded in: 0.0157 s

shape of the dataframe is: (2171, 5)


In [4]:
merged.head()

,event_index,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2,START_DATETIME_UTC,END_DATETIME_UTC,EVENT_DETAIL,EVENT_TYPE,KM_END,KM_START,KM_EVENT,step_duration,EMERGENCY_LANE,LANES,ROAD_TYPE
0,NaN,0,2018-09-01 01:00:00,443,97.889000,1.144099,97.080,98.698,2,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1
1,NaN,0,2018-09-01 03:15:00,443,154.788667,16.287509,137.530,169.890,3,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1
2,NaN,0,2018-09-01 12:45:00,443,136.091778,10.759666,124.586,163.418,18,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1
3,NaN,0,2018-09-01 19:15:00,443,136.916276,15.089653,114.878,169.890,58,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1
4,NaN,0,2018-09-02 09:45:00,443,135.476385,23.125778,82.518,169.890,26,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,1,2,1


In [5]:
merged = merged[['EVENT_TYPE', 'SPEED_AVG', 'ROAD_TYPE']].dropna().groupby(['EVENT_TYPE', 'ROAD_TYPE']).agg(['mean', 'std'])

In [6]:
merged.head()

SPEED_AVG           
                            mean        std
EVENT_TYPE ROAD_TYPE                       
Allarme    0          192.299270  15.924825
           1          210.330535  22.744094
           2          224.542896  22.765070
Barriere   0          150.232833  52.649715
           1          220.405290  25.721762

In [8]:
merged['AVG_SPEED_EVENT'] = merged['SPEED_AVG']['mean']
merged['STD_SPEED_EVENT'] = merged['SPEED_AVG']['std']

In [9]:
merged.columns = merged.columns.droplevel(level=1)

In [10]:
merged.head()

SPEED_AVG  SPEED_AVG  AVG_SPEED_EVENT  STD_SPEED_EVENT
EVENT_TYPE ROAD_TYPE                                                         
Allarme    0          192.299270  15.924825  192.299270       15.924825      
           1          210.330535  22.744094  210.330535       22.744094      
           2          224.542896  22.765070  224.542896       22.765070      
Barriere   0          150.232833  52.649715  150.232833       52.649715      
           1          220.405290  25.721762  220.405290       25.721762

In [16]:
merged.drop(['SPEED_AVG'], axis=1)

AVG_SPEED_EVENT  STD_SPEED_EVENT
EVENT_TYPE                  ROAD_TYPE                                  
Allarme                     0          192.299270       15.924825      
                            1          210.330535       22.744094      
                            2          224.542896       22.765070      
Barriere                    0          150.232833       52.649715      
                            1          220.405290       25.721762      
                            2          205.618223       21.516906      
Calamita_naturale           0          152.238971       19.998882      
                            1          214.233770       29.078995      
Gestione_viabilita          0          182.089812       26.744971      
                            1          175.826420       33.926805      
                            2          167.513055       20.960448      
Manutenzione_opere_in_verde 0          194.854631       32.224656      
                            1          214.672641       23.010298      
                            2          210.370597       15.464345      
Meteo                       0          144.279724       19.480299      
                            1          201.330924       9.289098       
Opera_arte                  0          173.089851       28.029674      
                            1          249.328072       45.670760      
Ostacolo_in_carreggiata     0          169.289942       27.424423      
                            1          181.542191       41.959591      
                            2          198.557223       24.603791      
Pavimentazione              0          160.120698       25.422131      
                            1          202.181883       27.063659      
                            2          209.986500       11.180272      
Regimazione_delle_acque     0          140.226846       34.969968      
Segnaletica_orizzontale     0          180.764405       29.368051      
                            1          205.827888       23.252339      
                            2          214.947729       10.542492      
Segnaletica_verticale       0          190.955954       30.552807      
                            1          223.754818       17.966613      
Veicolo_in_avaria           0          198.058325       33.866733      
                            1          190.523032       37.046478      
                            2          218.374561       21.796658      
extended_accident           0          177.212359       37.878365      
                            1          173.860189       49.197170      
                            2          166.280205       37.107195